In [3]:
#import libraries
# This ensures visualizations are plotted inside the notebook
%matplotlib inline
import io
import os              # This provides several system utilities
import pandas as pd    
import seaborn as sns 
import numpy as np
import geopandas as gpd
from shapely.geometry import Point
import matplotlib
import matplotlib.pyplot as plt 
from cenpy import products
import cenpy
import scipy.stats  as stats # low-level stats & probability
import statsmodels.formula.api as smf # high-level stats
import requests
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
import statsmodels.api as sm

In [4]:
#reproduce paper data
austria = pd.read_csv('data/AT_Austria.csv')
austria.head()

,Origin,Destination,Data,Oi,Dj,Dij,Offset,beta,OrigAT11,OrigAT12,...,DestAT12,DestAT13,DestAT21,DestAT22,DestAT31,DestAT32,DestAT33,DestAT34,Oi2007,Dj2007
0,AT11,AT11,0,4016,5146,1.000000e-300,1.000000e-300,1,0,0,...,0,0,0,0,0,0,0,0,4320,5452
1,AT11,AT12,1131,4016,25741,1.030018e+02,1.000000e+00,1,0,0,...,1,0,0,0,0,0,0,0,4320,27169
2,AT11,AT13,1887,4016,26980,8.420467e+01,1.000000e+00,1,0,0,...,0,1,0,0,0,0,0,0,4320,28710
3,AT11,AT21,69,4016,4117,2.208119e+02,1.000000e+00,1,0,0,...,0,0,1,0,0,0,0,0,4320,4354
4,AT11,AT22,738,4016,8634,1.320075e+02,1.000000e+00,1,0,0,...,0,0,0,1,0,0,0,0,4320,9069


In [5]:
austria = austria[austria['Origin']!=austria['Destination']]
flows = austria['Data'].values
Oi = austria['Oi'].values
Dj = austria['Dj'].values
Dij = austria['Dij'].values
Origin = austria['Origin'].values
Destination = austria['Destination'].values

In [14]:
input_features = np.vstack((Oi,Dj,Dij))
X=input_features.T
X = np.log(X)
y=flows.reshape(-1,1)
y = np.log(y)

In [10]:
# !pip install git+https://github.com/pysal/spint.git

In [12]:
from spint.gravity import Gravity
from spint.gravity import Production
from spint.gravity import Attraction
from spint.gravity import Doubly

In [15]:
gravity = Gravity(flows,Oi,Dj,Dij,'exp')

In [17]:
print(gravity.params)

[-8.01822841e+00  8.69316127e-01  8.91445153e-01 -6.22938370e-03]


In [18]:
Dij

array([103.001845,  84.204666, 220.811933, 132.00748 , 214.511814,
       246.933305, 390.85611 , 505.089539, 103.001845,  45.796272,
       216.994739, 129.878172, 140.706671, 201.232355, 343.50075 ,
       453.515594,  84.204666,  45.796272, 249.932874, 158.630661,
       186.420738, 244.108305, 387.61776 , 498.407152, 220.811933,
       216.994739, 249.932874,  92.407958, 151.777157,  92.894408,
       194.851669, 306.105825, 132.00748 , 129.878172, 158.630661,
        92.407958, 124.563096, 122.433524, 261.893783, 376.34667 ,
       214.511814, 140.706671, 186.420738, 151.777157, 124.563096,
        81.753652, 208.456383, 314.793199, 246.933305, 201.232355,
       244.108305,  92.894408, 122.433524,  81.753652, 145.076472,
       258.591197, 390.85611 , 343.50075 , 387.61776 , 194.851669,
       261.893783, 208.456383, 145.076472, 114.46325 , 505.089539,
       453.515594, 498.407152, 306.105825, 376.34667 , 314.793199,
       258.591197, 114.46325 ])

In [19]:
type(Oi)

numpy.ndarray

In [34]:
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, utils

In [78]:
class SpIntDataset(Dataset):
    def __init__(self, input_paras,target_flows):
        '''
        '''
        self.in_torch = torch.from_numpy(input_paras).t().view(-1,1,3).float()
        self.out_torch =  torch.from_numpy(target_flows).float().view(-1,1,1)
        
    def __len__(self):
        return len(self.in_torch)
    
    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()
        in_f = self.in_torch[idx,:,:]
        tar_flow = self.out_torch[idx,:,:]
        sample = {'input': in_f, 'output': tar_flow}
        return sample

In [13]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

In [21]:
class Net(nn.Module):
    def __init__(self):
        super(Net,self).__init__()
        self.fc1 = nn.Linear(3,8)
        self.fc2 = nn.Linear(8,1)
    
    def forward(self, x):
        x = F.sigmoid(self.fc1(x))
        x = self.fc2(x)
        return x
    
net = Net()

In [96]:
optimizer = optim.SGD(net.parameters(), lr=0.01)
criterion = nn.MSELoss()

In [23]:
flows

array([ 1131,  1887,    69,   738,    98,    31,    43,    19,  1633,
       14055,   416,  1276,  1850,   388,   303,   159,  2301, 20164,
        1080,  1831,  1943,   742,   674,   407,    85,   379,  1597,
        1608,   328,   317,   469,   114,   762,  1110,  2973,  1252,
        1081,   622,   425,   262,   196,  2027,  3498,   346,  1332,
        2144,   821,   274,    49,   378,  1349,   310,   851,  2117,
         630,   106,    87,   424,   978,   490,   670,   577,   546,
         569,    33,   128,   643,   154,   328,   199,   112,   587],
      dtype=int64)

In [6]:
input_features = np.vstack((Oi,Dj,Dij))
in_torch = torch.from_numpy(input_features)
in_torch = in_torch.t()
in_torch = in_torch.view(-1,1,3)
in_torch = in_torch.float()
in_torch[[0]]

NameError: name 'torch' is not defined

In [28]:
X=input_features.T

X = sm.add_constant(X)

In [29]:
Y=flows.reshape(-1,1)


In [30]:
# X = np.log(X)
# Y = np.log(Y)
# model = sm.OLS(Y,X)

In [31]:


#statsmodel GLM family possion/SPN
model = sm.GLM(Y, X, family=sm.families.Poisson())

In [33]:
results = model.fit()
results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                 Generalized Linear Model Regression Results                  
==============================================================================
Dep. Variable:                      y   No. Observations:                   72
Model:                            GLM   Df Residuals:                       68
Model Family:                 Poisson   Df Model:                            3
Link Function:                    log   Scale:                          1.0000
Method:                          IRLS   Log-Likelihood:                -8629.7
Date:                Tue, 03 Dec 2019   Deviance:                       16679.
Time:                        14:44:22   Pearson chi2:                 1.69e+04
No. Iterations:                     6                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const          6.4390      0.014    458.966      0.000       6.411       6.466
x1          6.572e-05   3.63e-07    181.091      0.000     6.5e-05    6.64e-05
x2          6.976e-05   3.64e-07    191.478      0.000     6.9e-05    7.05e-05
x3            -0.0066   4.73e-05   -138.504      0.000      -0.007      -0.006
==============================================================================
"""

In [20]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

In [21]:
y_test.shape

(15, 1)

In [22]:
#activation functions and their gradient functions
def sigmoid(X):
    return 1/(1+np.exp(-X))

def sigmoid_grad(X):
    return sigmoid(X) * (1 - sigmoid(X))

def tanh(z):
    return np.tanh(z)

def tanh_grad(z):
     return 1 - np.tanh(z) ** 2

def ReLU(z):
    return np.clip(z, 0, np.inf)

def ReLU_grad(z):
    return (z > 0).astype(int)

def affine(X,slope=1,intercept=0):
     return slope * X + intercept
    
def affine_grad(X,slope=1,intercept=0):
    return slope * np.ones_like(X)

In [23]:
#define neural network model
class NeuralNetwork:
    def __init__(self, input_dim, output_dim=1,hidden_dim = 4,lr=0.005):
        #init weights
        self.weights1   = np.random.rand(input_dim+1,hidden_dim) 
        self.weights2   = np.random.rand(hidden_dim,output_dim)                 
        #set learning rate
        self.lr         = lr
      
    def print_w(self):
        '''print weight to inspect the current values of network'''  
        print('print_weights ------------>')
        print(self.weights1)
        print(self.weights2)
        
    def feedforward(self,X):
        X = np.hstack((X,np.ones((X.shape[0],1))))
        self.layer1 = affine(np.dot(X, self.weights1))
        self.output = affine(np.dot(self.layer1, self.weights2))
        
    def backprop(self,X, Y):
        X = np.hstack((X,np.ones((X.shape[0],1))))
        # application of the chain rule to find derivative of the loss function with respect to weights2 and weights1
        d_weights2 = np.dot(self.layer1.T, (2*(Y - self.output) * affine_grad(np.dot(self.layer1, self.weights2))))
        d_weights1 = np.dot(X.T,  \
                            (np.dot(2*(Y - self.output) * affine_grad(np.dot(self.layer1, self.weights2)), self.weights2.T)\
                             * affine_grad(np.dot(X, self.weights1))))

        # update the weights with the derivative (slope) of the loss function multiply learning rate
        self.weights1 += d_weights1*self.lr
        self.weights2 += d_weights2*self.lr
    
    def test(self,X):
        '''get predicted values for any input data'''
        X = np.hstack((X,np.ones((X.shape[0],1))))
        hidden_layer1 = affine(np.dot(X, self.weights1))
        return affine(np.dot(hidden_layer1, self.weights2))
        
    def train(self,X,Y,num_train_iterations):
        '''train model with X and Y for num_train_iterations times'''
        print('training  ---------------->')
        for iteration in range(num_train_iterations): 
            self.feedforward(X) 
            self.backprop(X,Y)
            #print interim MSE
            if iteration % 100 == 0:
                mse = np.mean((self.output - Y)**2)
                print("Epoch ", iteration, "MSE: ", mse)
                

In [24]:
batch_size = 3

#initialize network with fixed output dim of 1
neural_network = NeuralNetwork(X_train.shape[1],1,lr=1e-3)

for index in range(0,X_train.shape[0],batch_size):
    
    
    #get batch X and Y
    batch_X=X_train[index:min(index+batch_size,X_train.shape[0]),:]
    batch_Y=y_train[index:min(index+batch_size,y_train.shape[0])]
    
    #train model with batch
    neural_network.train(batch_X,batch_Y,500)
    
    #print final state of weights
    neural_network.print_w()

    # Test the neural network with new test data. 
    #get predicted y
    y_pred = neural_network.test(X_test)
    #compare predicted y and groundtruth 
    print('predicted data ----------->')
    print(y_pred)
    print('real data ---------------->')
    print(y_test)
    #calculate MSE
    mse = np.mean((y_test - y_pred)**2)
    print('MSE on test data --------->')
    print(mse)

training  ---------------->
Epoch  0 MSE:  1486.397958130395
Epoch  100 MSE:  nan
Epoch  200 MSE:  nan
Epoch  300 MSE:  nan
Epoch  400 MSE:  nan
print_weights ------------>
[[nan nan nan nan]
 [nan nan nan nan]
 [nan nan nan nan]
 [nan nan nan nan]
 [nan nan nan nan]]
[[nan]
 [nan]
 [nan]
 [nan]]
predicted data ----------->
[[nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]]
real data ---------------->
[[4.58496748]
 [6.30261898]
 [6.98471632]
 [7.03085748]
 [5.79301361]
 [7.20711886]
 [6.03068526]
 [7.99732682]
 [7.52294092]
 [6.44571982]
 [5.61312811]
 [4.73619845]
 [9.55073348]
 [7.67042852]
 [3.4339872 ]]
MSE on test data --------->
nan
training  ---------------->
Epoch  0 MSE:  nan
Epoch  100 MSE:  nan
Epoch  200 MSE:  nan
Epoch  300 MSE:  nan
Epoch  400 MSE:  nan
print_weights ------------>
[[nan nan nan nan]
 [nan nan nan nan]
 [nan nan nan nan]
 [nan nan nan nan]
 [nan nan nan nan]]
[[nan]
 [nan]
 [nan]
 [nan]]
predicted dat

In [25]:
kf = KFold(n_splits=6)
kf.get_n_splits(X)
batch_size = 3
for train_index, test_index in kf.split(X):

    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]
    #initialize network with fixed output dim of 1
    neural_network = NeuralNetwork(X_train.shape[1],1,lr=1e-3)

    for index in range(0,X_train.shape[0],batch_size):


        #get batch X and Y
        batch_X=X_train[index:min(index+batch_size,X_train.shape[0]),:]
        batch_Y=y_train[index:min(index+batch_size,y_train.shape[0])]

        #train model with batch
        neural_network.train(batch_X,batch_Y,500)

        #print final state of weights
        neural_network.print_w()

        # Test the neural network with new test data. 
        #get predicted y
        y_pred = neural_network.test(X_test)
        #compare predicted y and groundtruth 
        print('predicted data ----------->')
        print(y_pred)
        print('real data ---------------->')
        print(y_test)
        #calculate MSE
        mse = np.mean((y_test - y_pred)**2)
        print('MSE on test data --------->')
        print(mse)

training  ---------------->
Epoch  0 MSE:  1667.6191438036683
Epoch  100 MSE:  nan
Epoch  200 MSE:  nan
Epoch  300 MSE:  nan
Epoch  400 MSE:  nan
print_weights ------------>
[[nan nan nan nan]
 [nan nan nan nan]
 [nan nan nan nan]
 [nan nan nan nan]
 [nan nan nan nan]]
[[nan]
 [nan]
 [nan]
 [nan]]
predicted data ----------->
[[nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]]
real data ---------------->
[[7.03085748]
 [7.54274355]
 [4.2341065 ]
 [6.60394382]
 [4.58496748]
 [3.4339872 ]
 [3.76120012]
 [2.94443898]
 [7.39817409]
 [9.55073348]
 [6.03068526]
 [7.15148546]]
MSE on test data --------->
nan
training  ---------------->
Epoch  0 MSE:  nan
Epoch  100 MSE:  nan
Epoch  200 MSE:  nan
Epoch  300 MSE:  nan
Epoch  400 MSE:  nan
print_weights ------------>
[[nan nan nan nan]
 [nan nan nan nan]
 [nan nan nan nan]
 [nan nan nan nan]
 [nan nan nan nan]]
[[nan]
 [nan]
 [nan]
 [nan]]
predicted data ----------->
[[nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan

C:\ProgramData\Anaconda3\envs\geo\lib\site-packages\ipykernel_launcher.py:30: RuntimeWarning: invalid value encountered in add
C:\ProgramData\Anaconda3\envs\geo\lib\site-packages\ipykernel_launcher.py:31: RuntimeWarning: invalid value encountered in add


 300 MSE:  nan
Epoch  400 MSE:  nan
print_weights ------------>
[[nan nan nan nan]
 [nan nan nan nan]
 [nan nan nan nan]
 [nan nan nan nan]
 [nan nan nan nan]]
[[nan]
 [nan]
 [nan]
 [nan]]
predicted data ----------->
[[nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]]
real data ---------------->
[[7.03085748]
 [7.54274355]
 [4.2341065 ]
 [6.60394382]
 [4.58496748]
 [3.4339872 ]
 [3.76120012]
 [2.94443898]
 [7.39817409]
 [9.55073348]
 [6.03068526]
 [7.15148546]]
MSE on test data --------->
nan
training  ---------------->
Epoch  0 MSE:  nan
Epoch  100 MSE:  nan
Epoch  200 MSE:  nan
Epoch  300 MSE:  nan
Epoch  400 MSE:  nan
print_weights ------------>
[[nan nan nan nan]
 [nan nan nan nan]
 [nan nan nan nan]
 [nan nan nan nan]
 [nan nan nan nan]]
[[nan]
 [nan]
 [nan]
 [nan]]
predicted data ----------->
[[nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]]
real data ---------------->
[[7.03085748]
 [7.54274355]
 [4.2341065 ]
 [

In [77]:
in_torch[[1,2,3],:,:]

tensor([[[ 4016.0000, 26980.0000,    84.2047]],

        [[ 4016.0000,  4117.0000,   220.8119]],

        [[ 4016.0000,  8634.0000,   132.0075]]])

In [79]:
out_torch = torch.from_numpy(flows).float()
out_torch = out_torch.view(-1,1,1)


In [80]:
spint_dataset = SpIntDataset(input_features,flows)

In [98]:
dataloader = DataLoader(spint_dataset, batch_size=1,
                        shuffle=True)

In [ ]:
#train the network

for epoch in range(200):  # loop over the dataset multiple times

    running_loss = 0.0
    for i, data in enumerate(dataloader, 0):
        
        in_f,out_f = data['input'], data['output']
        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = net(in_f)
        loss = criterion(outputs, out_f)
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
        if i % 20 == 19:
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, running_loss / 2000))
            running_loss = 0.0

print('Finished Training')

In [101]:
PATH = './austria_ann.pth'
torch.save(net.state_dict(), PATH)

In [103]:
dataiter = iter(dataloader)
data = dataiter.next()
data

{'input': tensor([[[29142.0000,  4117.0000,   249.9329]]]),
 'output': tensor([[[1080.]]])}

In [104]:
net = Net()
net.load_state_dict(torch.load(PATH))

<All keys matched successfully>

In [105]:
outputs = net(data['input'])

In [33]:
out_torch.shape

torch.Size([1, 72])

In [46]:
input_test = torch.randn(1, 1, 3)

target_test = torch.randn(1,1,1)
target_test

tensor([[[-0.4147]]])

In [69]:
optimizer.zero_grad()   # zero the gradient buffers


In [ ]:

output = net(in_torch)
output


In [71]:
loss = criterion(output, out_torch)


In [72]:
loss.backward()
loss.item()

9391175.0

In [73]:
optimizer.step()    # Does the update

In [ ]:
#train the network

for epoch in range(2):  # loop over the dataset multiple times

    running_loss = 0.0
    for i, data in enumerate(dataloader, 0):
        

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = net(in_torch)
        loss = criterion(outputs, out_torch)
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
        if i % 2000 == 1999:    # print every 2000 mini-batches
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, running_loss / 2000))
            running_loss = 0.0

print('Finished Training')